In [19]:
# libraries
import random
import math
import scipy
from scipy import linalg
import numpy as np
import pandas as pd

In [20]:
# data 
def Data():
    # membaca data
    file_name = 'datasets.xlsx'
    data_training = pd.read_excel(file_name, sheet_name='training', usecols=range(1, 14))
    data_testing = pd.read_excel(file_name, sheet_name='testing', usecols=range(1, 14))
    train_target = pd.read_excel(file_name, sheet_name='training', usecols=[14])
    test_target = pd.read_excel(file_name, sheet_name='testing', usecols=[14])

    # mengubah data dalam bentuk matriks
    Data.dt_training = data_training.to_numpy()
    Data.dt_testing = data_testing.to_numpy()
    Data.dt_target_training = train_target.to_numpy()
    Data.dt_target_testing = test_target.to_numpy()
    

In [21]:
def Hidden_layer(input_weights, biases, n_hidden_node, data_input):
    # inisialisasi input weight
    input_weight = input_weights.reshape(n_hidden_node, 13)

    # inisialisasi bias
    bias = biases

    # transpose input weight
    transpose_input_weight = np.transpose(input_weight)

    # matriks output hidden layer
    hidden_layer = []
    for data in range(len(data_input)):

        # perkalian data input dengan input weight transpose
        h = np.matmul(data_input[data], transpose_input_weight)
        
        # penambahan dengan bias
        h_output = np.add(h, bias)
        hidden_layer.append(h_output)
    
    return hidden_layer

In [22]:
# aktivasi hasil keluaran hidden layer
def Activation(hidden_layer):
    for row in range(len(hidden_layer)):
        for col in range(len(hidden_layer[row])):
            hidden_layer[row][col] = 1 / (1 + np.exp((hidden_layer[row][col] * (-1))))
    activation = hidden_layer
    
    return activation

In [23]:
# matriks moore penrose pseudo-inverse menggunakan SVD
def Pseudoinverse(hidden_layer):
    h_pseudo_inverse = scipy.linalg.pinv2(hidden_layer, cond=None, rcond=None, return_rank=False, check_finite=True)
    
    return h_pseudo_inverse

In [24]:
# menghitung output weight
def Output_weight(pseudo_inverse, target):
    beta = np.matmul(pseudo_inverse, target)
    
    return beta

In [25]:
# menghitung hasil prediksi pada data testing
def Target_output(testing_hidden_layer, output_weight):
    target = np.matmul(testing_hidden_layer, output_weight)
    # memetakan matriks target pada klasifikasi
    prediction = []
    for result in range(len(target)):
        dist_target_0 = abs(target[result] - 0)
        dist_target_1 = abs(target[result] - 1)
        min_dist = min(dist_target_0, dist_target_1)
        if min_dist == dist_target_0:
            predict = 0
        elif min_dist == dist_target_1:
            predict = 1
        prediction.append(predict)
        
    return prediction

In [26]:
# inisialisasi partikel / posisi partikel (input weight & bias)
def Particle(n_inputWeights, n_biases):
    # inisialisasi input weight
    input_weights = []
    for input_weight in range(0, n_inputWeights):
        input_weights.append(round(random.uniform(-1.0, 1.0), 2))

    # inisialisasi bias
    biases = []
    for bias in range(0, n_biases):
        biases.append(round(random.random(), 2))

    return input_weights + biases

In [27]:
# inisialisasi kecepatan awal 
def Velocity_0(n_particles):
    return [0] * n_particles

In [28]:
# evaluasi akurasi
def Evaluate(actual, prediction):
    true = 0
    for i in range(min(len(actual), len(prediction))):
        if actual[i] == prediction[i]:
            true += 1
    # akurasi
    accuracy = round(((true / len(prediction)) * 100), 2) 
    
    return accuracy

In [29]:
# mendapatkan pBest
def Pbest(particles, fitness):
    fitness = np.expand_dims(fitness, axis=1)
    pbest = np.hstack((particles, fitness))
    
    return pbest

In [30]:
# membandingkan pbest ke t dan pbest ke t+1
def Comparison(pbest_t, pbest_t_1):
    for i in range(min(len(pbest_t), len(pbest_t_1))):
        if pbest_t[i][-1] > pbest_t_1[i][-1]:
            pbest_t_1[i] = pbest_t[i]
        else:
            pbest_t_1[i] = pbest_t_1[i]
    
    return pbest_t_1

In [31]:
# mendapatkan partikel terbaik dalam suatu populasi
def Gbest(particles, fitness):
    # fitness / akurasi terbaik
    best_fitness = np.amax(fitness)
    
    # partikel dengan fitness terbaik
    particle = fitness.index(best_fitness)
    best_particle = particles[particle]

    # gbest
    gbest = np.hstack((best_particle, best_fitness))

    return gbest  

In [32]:
# update kecepatan
def Velocity_update(pbest, gbest, w, c1, c2, particles, velocity):

    # mencari batas tiap fitur
    interval = []
    for i in range(len(particles[0])):
        x_max = np.amax(np.array(particles)[:, i])
        x_min = np.amin(np.array(particles)[:, i])
        k = round(random.random(), 1)
        v_max_i = np.array(((x_max - x_min)/2) * k)
        v_min_i = np.array(v_max_i * -1)
        intvl = np.hstack((v_min_i, v_max_i))
        interval.append(intvl)

    # update kecepatan
    r1 = round(random.random(), 1)
    r2 = round(random.random(), 1)
    for i in range(min(len(particles), len(velocity), len(pbest), len(gbest))):
        for j in range(min(len(particles[i]) - 1, len(pbest[i]) - 1)):
            velocity[i] = (w * velocity[i]) + (c1 * r1 * (pbest[i][j] - particles[i][j])) + (c2 * r2 * (gbest[i] - particles[i][j]))
    
    return velocity
    

In [33]:
# update posisi partikel
def Position_update(current_position, velocity_update):
    for i in range(min(len(current_position), len(velocity_update))):
        for j in range(len(current_position[i])):
            current_position[i][j] = (current_position[i][j] + velocity_update[i])
    
    return current_position

In [34]:
# fungsi ELM
def Elm(particles, n_input_weights, n_hidden_node):
    # fitness = akurasi
    fitness = []

    for i in range(len(particles)):
        # proses elm
        #-----------------training---------------------#
           
        # input weight
        input_weights = np.array(particles[i][0:n_input_weights])
          
        # bias
        biases = np.array(particles[i][n_input_weights:len(particles[i])])

        # menghitung matriks keluaran hidden layer pada data training
        hidden_layer_training = Hidden_layer(input_weights, biases, n_hidden_node, Data.dt_training)

        # aktivasi hasil keluaran hidden layer data training
        activation_training = Activation(hidden_layer_training) 

        # matriks moore penrose 
        pseudo_training = Pseudoinverse(activation_training)

        # menghitung output weight pada data training
        output_training = Output_weight(pseudo_training, Data.dt_target_training)

        #-----------------testing--------------------#

        # menghitung matriks keluaran hidden layer pada data testing
        hidden_layer_testing = Hidden_layer(input_weights, biases, n_hidden_node, Data.dt_testing)

        # aktivasi matriks keluaran hidden layer data testing
        activation_testing = Activation(hidden_layer_testing)

        # menghitung hasil prediksi pada data testing
        prediction = Target_output(hidden_layer_testing, output_training)

        # akurasi
        accuracy = Evaluate(Data.dt_target_testing, prediction)
        fitness.append(accuracy)

    return fitness

In [35]:
def Run():

    # inisialisasi PSO
    fitures = 13
    n_hidden_node = 5 # jumlah hidden node / partikel bias
    n_input_weights = n_hidden_node * fitures # partikel input weight
    population = 100 # jumlah populasi pada tiap iterasi
    max_iter = 3 # iterasi maksimum
    w = 0.5 # bobot inersia
    c1 = 1 # kontansta kecepatan 1
    c2 = 1 # konstanta kecepatan 2

    # data
    Data()

    # inisialisasi kecepatan awal
    velocity_t = Velocity_0(population)

    # inisialisasi posisi awal
    particles = []
    # perulangan sebanyak populasi
    for pop in range(population):
        particle = Particle(n_input_weights, n_hidden_node)
        particles.append(particle)

    # fitness tiap partikel = akurasi elm
    fitness_t = Elm(particles, n_input_weights, n_hidden_node)

    # inisialisasi Pbest
    pbest_t = Pbest(particles, fitness_t)

    # inisialisasi Gbest
    gbest_t = Gbest(particles, fitness_t)

    for iteration in range(max_iter):

        # update kecepatan
        velocity_t_1 = Velocity_update(pbest_t, gbest_t, w, c1, c2, particles, velocity_t)

        # update posisi partikel
        particles_t_1 = Position_update(particles, velocity_t_1)

        # elm
        fitness_t_1 = Elm(particles_t_1, n_input_weights, n_hidden_node)

        # update pbest
        pbest_t_1 = Pbest(particles_t_1, fitness_t_1)
        pbest_t_1 = Comparison(pbest_t, pbest_t_1)

        # update gbest
        gbest_t_1 = Gbest(particles_t_1, fitness_t_1)

        #------------------------#
        pbest_t = pbest_t_1
        gbest_t = gbest_t_1
        particles = particles_t_1
        velocity_t = velocity_t_1
        
    print('Input Weights')
    print(gbest_t_1[0:n_input_weights])
    print('')
    print('Biases')
    print(gbest_t_1[n_input_weights:len(gbest_t_1) - 1])
    print('')
    print('Accuracy')
    print(gbest_t_1[-1])

In [36]:
if __name__ == '__main__':
    Run()

Input Weights
[-0.2   0.32  0.28 -0.09 -0.68  0.77 -0.95  0.52 -0.35 -0.64  0.04  0.46
 -0.41 -0.71  0.62 -0.26 -0.68 -0.28 -0.89  0.2   0.27  0.13 -0.8  -0.74
 -0.11 -0.62 -0.34 -0.11  0.76  0.38  0.3  -0.93 -0.59  0.96 -0.79 -0.05
  0.12 -0.3  -0.47  0.5   0.24  0.92  0.51  0.84 -0.85 -0.42  0.04  0.61
  0.6  -0.22 -0.61  0.32  0.55  0.84 -0.15 -0.65 -0.16 -0.73  0.38  0.82
 -0.2   0.7   0.53  0.47 -0.11]

Biases
[0.29 0.47 0.45 0.18 0.83]

Accuracy
81.97
